In [ ]:
# 이전에 split했었던 문서에 embed 작업을 할것이다.
# 먼저 embedding model에 대해 살펴보자 

from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

# print(embeddings.embed_query("Hi")) # Hi에 대한 벡터를 얻게됨
vector = embeddings.embed_query("Hi")

print(len(vector)) # "Hi"는 1536개의 차원을 가지고 있음


In [ ]:
vector = embeddings.embed_documents([
    "Hi",
    "how",
    "are",
    "you longer sentences because"
])

# print(len(vector)) # 4개의 문장에 대해 벡터를 만들었기 때문에 4개의 벡터를 얻게됨
print(len(vector), len(vector[0])) # 첫번째 문자 "Hi"에 대한 벡터는 1536개의 차원을 가지고 있음
# print(vector)


In [1]:
# vector store란 일종의 데이터베이스라고 생각하면 된다.
# 벡터 공간에서 검색을 할 수 있게 해준다.
# 우리가 벡터들을 만들고 나서, 그것들을 캐시해주고 vector store에 그 벡터들을 넣어주면 우리가 검색할 수 있다.
# 관련있는 문서들만 찾아낼수 있다는 의미
# 랭체인은 다양한 Vector Store를 지원한다. 그 중일부는 cloud에 있는데 유료도 있고 무료도 있다.
# 여기선 오픈소스 하나를 사용할건데 cloud환경이 아닌 로컬에서 직접 실행할것이고 이름은 Chroma이다.
# Chroma를 이용해 우리 문서들을 계산해서 얻어낸 vector들 속에서 검색을 할 수 있다.
# 바로 실습해보자

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter 
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings # 분할된 문서와 openAI embeddings model을 전달해야된다.
from langchain.vectorstores import Chroma # 크로마 불러오기
from langchain.storage import LocalFileStore

# 임베딩을 캐시하기 위해 캐시 디렉토리를 생성
cache_dir = LocalFileStore("./.cache/") # .cache 디렉토리를 생성됨 (여기에 임베딩된 데이터들이 저장됨)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", # 문단 기준으로 분할
    chunk_size=600, # 최대 글자개수 600인 문서로 나눔 (separator가 없으면 좀 더 커질수도 있고 작아질 수도 있음)
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/test.txt")

# 문서 분할
docs = loader.load_and_split(text_splitter=splitter)

# 임베딩 생성(벡터화)
embeddings = OpenAIEmbeddings()

# 임베딩을 저장할 장소인 cache_dir도 전달
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

# 분할된 문서와 openAI embeddings model을 전달해야된다.
# 이렇게 임베딩을 캐싱하게되면 다음에 호출할때에 캐싱되어있는 임베딩을 전달하게 되어 요금을 아낄 수 있다.
vector_store = Chroma.from_documents(
    docs,
    # embedding=embeddings,
    cached_embeddings # 처음엔 캐시에 임베딩이 존재하는지 확인. 재 호출시 캐싱되어있는 임베딩을 전달
)
# 실행하면 바로 API요금이 나가니 너무 긴 문서로 테스트 하지 말자. 재시작도 하지 말고 밑에 코드에서 진행 (캐싱하면 상관없음)

In [2]:
# 벡터 공간에서 검색을 시작할 수 있다.
# 우리의 문서들이 벡터로 변환되고 그 벡터 공간을 검색 할 수 있게 된다.
results = vector_store.similarity_search("뭘 설명하고 있습니까?")

# 검색을 하게되면 그 질문(query)과 관련있는 문서들을 반환하게 된다.
# 받은 문서들이 너무 크면 엄청 큰 문서들이 LLM에게 전달될것이고, 그만큼 돈을 지출하게 된다.
# 그래서 작은 부분으로 분할(split)해서 전달하는 방법은 좋은 아이디어다. (너무 작게 분할하면 문백과 의미가 사라질수 있기 때문에 적당히 분할해야된다.)

# len(results)
print(results)

[Document(page_content='Measureless sand ... interminable sand....\nThe smooth hide of that yellow lion, Earth, Ruffled a little and was dark again Beneath the descending torrents of the night, Plunging like cobalt from the cliffs of the sky, Blotting the stiff wedge of each pyramid With the slow gurgle of a rising wave, A wave burning with stars....\nThe Sphinx alone Couched on her forepaws like a sleepy hound Under the weight of a caress of rock And smiled her woman’s and chimera’s smile Inexorably, drowned with the savage dark.\nThe black tide filled the heavens up and ceased, A little tongueing flame ran on the sand Bright as a fire of paper, swift and light As a bird’s restless eyes. It rose. It bloomed, An angry dream before the Sphinx’s feet, The exhalation of a furious thought, Tall as the ghosts of Heaven’s battlements, The apparition that had once been Troy!\nA girl went out in the summer skies, (_The dice lie white for the throwing!_) A girl went out in the summer skies And 